<p style="font-size:20px;color:#12ABDB;text-indent:2em">A. Libraries import</p>

In [1]:
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from shapely.geometry import Polygon
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# move to the challenge folder
os.chdir("/content/gdrive/MyDrive/data_challenge_mines")

<p style="font-size:20px;color:#12ABDB;text-indent:2em">B. Loading images</p>

In [5]:
import cv2
import numpy as np

# Charger un ensemble d'images pour un groupe particulier (50 échantillons d'images de voitures)
pathimages = "datasets_train/train/"
images = []

path_annotrain = "datasets_train/train_annotation/_annotation.csv"
train_annotation = pd.read_csv(path_annotrain, index_col='im_name')

bbox_train_full = train_annotation.loc[
    :, ["x_min", "y_min", "x_max", "y_max"]
]

In [6]:
os.chdir("/content/gdrive/MyDrive/data_challenge_mines")
images = []
for name_img in os.listdir(pathimages):
  if train_annotation.loc[name_img]['class'] == 1:
    images.append((cv2.imread(pathimages+name_img),name_img))

In [10]:
def exportcadre(img, name_image):
  x1, y1, x2, y2 = bbox_train_full.loc[name_image]
  try :
    mask = img[int(y1):int(y2), int(x1):int(x2)]
  except:
    mask = img
  return mask

In [11]:
goodimages = []
for image,nameimage in images:
  test_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  test_image = exportcadre(test_image, nameimage)
  test_image = cv2.resize(test_image, (640,640))
  goodimages.append([test_image, nameimage])
len(goodimages)

2094

In [27]:
def findBestKeypoints(keyp, n=25):
  listRep = []
  for i,kp in enumerate(keyp):
    listRep.append(np.array([i,kp.response]))
  res = sorted(listRep, key=lambda x: -x[1])
  return np.array(res[0:n])[:,0]


def detection(images):
  sift = cv2.SIFT_create()
  Descriptors = []
  numero=0
  for image,nameimage in images:
    keypoints,descript = sift.detectAndCompute(image, None)
    bestkeypoints = findBestKeypoints(keypoints)
    descriptor = []
    for i in bestkeypoints:
      descriptor.extend(descript[int(i)])
    Descriptors.append(np.array(descriptor).reshape(1,3200))

    numero += 1
    if numero % 100 == 0:
      print(numero)

  all_descriptors = np.concatenate(Descriptors, axis = 0)
  return all_descriptors


from sklearn.cluster import KMeans

def entrainement(images, all_descriptors, num_cluster=100):
  kmeans = KMeans(n_clusters=num_cluster)
  kmeans.fit(all_descriptors)
  return kmeans

In [28]:
all_descriptors = detection(images)
kmeans = entrainement(goodimages, all_descriptors)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [29]:
from collections import Counter

# Compter combien de descripteurs sont dans chaque cluster
cluster_counts = Counter(kmeans.labels_)

# Afficher le nombre de descripteurs dans chaque cluster
print(cluster_counts)

Counter({7: 118, 29: 98, 42: 97, 63: 85, 12: 79, 28: 76, 59: 74, 25: 60, 34: 60, 4: 53, 76: 49, 11: 48, 5: 48, 31: 44, 99: 38, 41: 35, 50: 31, 61: 30, 21: 29, 58: 28, 9: 27, 98: 27, 82: 26, 2: 25, 69: 25, 80: 23, 62: 22, 92: 21, 15: 21, 73: 20, 66: 20, 67: 19, 46: 19, 49: 18, 6: 18, 48: 17, 24: 17, 86: 17, 57: 17, 43: 16, 18: 16, 16: 15, 26: 14, 97: 14, 94: 14, 14: 14, 85: 13, 35: 13, 19: 13, 51: 13, 0: 12, 36: 12, 23: 12, 47: 12, 64: 12, 44: 11, 54: 11, 56: 11, 40: 11, 38: 11, 17: 11, 90: 11, 95: 10, 37: 10, 75: 10, 39: 10, 32: 9, 83: 9, 72: 9, 81: 9, 20: 8, 3: 8, 68: 8, 84: 7, 22: 7, 91: 7, 78: 6, 65: 6, 87: 6, 88: 6, 89: 6, 33: 6, 60: 6, 53: 5, 8: 5, 96: 5, 70: 5, 1: 4, 55: 4, 27: 4, 71: 4, 74: 4, 52: 4, 79: 4, 45: 3, 77: 3, 10: 2, 30: 2, 93: 1, 13: 1})


In [ ]:
lbis = np.array(lbis)
lbis.min(), lbis.max(), lbis.mean()

(31, 8524, 1282.0062082139445)

<p style="font-size:20px;color:#12ABDB;text-indent:2em">E. Metric to be recommended to use</p>

In [ ]:
def local_MAPE(e_true_i, e_pred_i):
    """
    Calculate the Mean Absolute Percentage Error (MAPE) for the image i.

    Args:
    e_true_i (float): The true values representing the true gas emission.
    e_pred_i (float): The predicted values representing the predicted gas emission.

    Returns:
    float: MAPE score between 0 and 1.
    """
    if e_true_i != 0:
        return abs(e_true_i - e_pred_i) / e_true_i
    else:
        if e_pred_i == 0:
            return 1
        else:
            return 0
